In [5]:
import numpy as np

# ---------- ACTIVATION FUNCTIONS ----------
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1.0, 0.0)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

def linear(x):
    return x

def linear_derivative(x):
    return np.ones_like(x)

def swish(x):
    return x * sigmoid(x)

def swish_derivative(x):
    s = sigmoid(x)
    return s + x * s * (1 - s)

def softmax(x):
    exps = np.exp(x - np.max(x))
    return exps / np.sum(exps)


# ---------- ACTIVATION SELECTOR ----------
def get_activation(name):
    name = name.lower()
    if name == "sigmoid":
        return sigmoid, sigmoid_derivative
    elif name == "relu":
        return relu, relu_derivative
    elif name == "tanh":
        return tanh, tanh_derivative
    elif name == "linear":
        return linear, linear_derivative
    elif name == "swish":
        return swish, swish_derivative
    elif name == "softmax":
        return softmax, None
    else:
        raise ValueError("Unsupported activation function")


# ---------- USER INPUT ----------
n_inputs = int(input("Enter number of inputs: "))
X = np.array(list(map(float, input(f"Enter {n_inputs} input values: ").split())))

n_layers = int(input("Enter number of layers (hidden + output): "))

layer_sizes = []
for i in range(n_layers):
    layer_sizes.append(int(input(f"Enter number of neurons in layer {i+1}: ")))

hidden_act_name = input(
    "\nChoose hidden activation (sigmoid/relu/tanh/linear/swish): "
)
output_act_name = input(
    "Choose output activation (sigmoid/relu/tanh/linear/softmax): "
)

hidden_act, hidden_der = get_activation(hidden_act_name)
output_act, output_der = get_activation(output_act_name)

weights = []
biases = []

prev_size = n_inputs
for i, size in enumerate(layer_sizes):
    print(f"\nEnter weights for layer {i+1} ({size} x {prev_size}):")
    W = np.array([list(map(float, input().split())) for _ in range(size)])
    b = np.array(list(map(float, input(f"Enter {size} biases for layer {i+1}: ").split())))

    weights.append(W)
    biases.append(b)
    prev_size = size

# ---------- TARGET ----------
if output_act_name.lower() == "softmax":
    target = np.array(list(map(float, input(
        f"\nEnter {layer_sizes[-1]} target values : "
    ).split())))
else:
    target = float(input("\nEnter target output: "))

lr = float(input("Enter learning rate: "))

# ---------- TRAINING ----------
epoch = 1
max_epochs = 10000
error = float('inf')

while epoch <= max_epochs:

    activations = [X]
    nets = []

    # ===== FORWARD =====
    for i in range(len(weights)):
        net = np.dot(weights[i], activations[-1]) + biases[i]
        nets.append(net)

        if i == len(weights) - 1:
            activations.append(output_act(net))
        else:
            activations.append(hidden_act(net))

    output = activations[-1]

    # ===== LOSS & DELTA =====
    if output_act_name.lower() == "softmax":
        error = -np.sum(target * np.log(output + 1e-9))
        delta = output - target   # softmax + cross-entropy
    else:
        error = target - output[0]
        delta = error * output_der(nets[-1])

    deltas = [delta]

    # ===== BACKPROP =====
    for i in range(len(weights) - 2, -1, -1):
        delta = np.dot(weights[i + 1].T, deltas[-1]) * hidden_der(nets[i])
        deltas.append(delta)

    deltas.reverse()

    # ===== UPDATE =====
    for i in range(len(weights)):
        weights[i] += lr * np.outer(deltas[i], activations[i])
        biases[i] += lr * deltas[i]

    print(f"Epoch {epoch}: Error = {error:.6f}")

    if abs(error) < 1e-6:
        break

    epoch += 1

print("\nTraining complete.")


Enter number of inputs:  3
Enter 3 input values:  3 5 2
Enter number of layers (hidden + output):  3
Enter number of neurons in layer 1:  2
Enter number of neurons in layer 2:  2
Enter number of neurons in layer 3:  1

Choose hidden activation (sigmoid/relu/tanh/linear/swish):  relu
Choose output activation (sigmoid/relu/tanh/linear/softmax):  softmax



Enter weights for layer 1 (2 x 3):


 0.1 -0.9 -0.8
 0.7 0.6 0.8
Enter 2 biases for layer 1:  0.1 0.6



Enter weights for layer 2 (2 x 2):


 0.2 0.9
 -0.4 0.2
Enter 2 biases for layer 2:  0.5 1



Enter weights for layer 3 (1 x 2):


 0.2 0.4
Enter 1 biases for layer 3:  1

Enter 1 target values :  1
Enter learning rate:  0.63


Epoch 1: Error = -0.000000

Training complete.
